# Objective

This project showcases an agentic Retrieval-Augmented Generation (RAG) system built using CrewAI agents to simulate an AI-powered technical support workflow. The goal is to provide users with accurate, contextual, and well-structured responses based on two available cloud documentation sources: Azure and AWS.

The support system is composed of three collaborating agents:

1. Router Agent – Interprets the user's query and intelligently selects the appropriate cloud documentation source (Azure or AWS).

2. Senior Support Agent – Searches through the selected documentation to extract relevant information and draft a response with the required steps or guidance.

3. Senior Support Quality Assurance Specialist Agent – Reviews the response for completeness, clarity, and accuracy. It then reformats the response into a well-structured format resembling an email (although no actual email is sent).

This setup simulates a real-world technical support workflow with modular, human-like reasoning and communication capabilities enabled through multi-agent collaboration.

Libraries used:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from crewai import Agent, Task, Crew

In [ ]:
import os
from google.colab import userdata

openai_api_key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

# Agents

In [ ]:
router_agent = Agent(
    role="Router Agent",
    goal="Identify whether the user inquiry relates to Azure or AWS documentation and route accordingly",
    backstory=(
        "You're an expert in cloud services who understands both Azure and AWS. "
        "When given an inquiry, your job is to decide whether it relates to Azure or AWS, "
        "and then choose the most relevant documentation URL."
    ),
    allow_delegation=False,
    verbose=True
)

In [ ]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at CrewAITech and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True
)

In [ ]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAITech and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	allow_delegation=True,
	verbose=True
)

# Tools

In [ ]:
from crewai_tools import ScrapeWebsiteTool

In [ ]:
scrape_tool = ScrapeWebsiteTool()

# Tasks

In [ ]:
routing_task = Task(
    agent=router_agent,
    description=(
        "Given this user inquiry within single quotes: '{inquiry}', determine whether it relates to Azure or AWS. "
        "Return the best documentation URL: either 'https://azure.microsoft.com/en-in/get-started/' "
        "or 'https://docs.aws.amazon.com/'."
    ),
    expected_output="A single valid URL related to the inquiry"
)

In [ ]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
        "Use the documentation from the URL provided by the routing_task to find the correct answer to the customer's inquiry: '{inquiry}'. "
        "Provide a complete and friendly response based only on that documentation."
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[scrape_tool],
    context=[routing_task],
    process_context=True,
    agent=support_agent,
)

In [ ]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Customer name: {person} from {customer}\n"
        "Inquiry: {inquiry}\n\n"
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


# Creating and Running the Crew

In [ ]:
crew = Crew(
  agents=[router_agent,support_agent, support_quality_assurance_agent],
  tasks=[routing_task,inquiry_resolution, quality_assurance_review],
  verbose=2,
  memory=True
)

In [ ]:
inputs = {
    "customer": "XYZ Corporation",
    "person": "Anusha",
    "inquiry": "How to set up up a Windows virtual machine in Azure using the Azure portal?"
}
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Router Agent
 [INFO]: == Starting Task: Given this user inquiry within single quotes: 'How to set up up a Windows virtual machine in Azure using the Azure portal?', determine whether it relates to Azure or AWS. Return the best documentation URL: either 'https://azure.microsoft.com/en-in/get-started/' or 'https://docs.aws.amazon.com/'.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: https://azure.microsoft.com/en-in/get-started/

> Finished chain.
 [DEBUG]: == [Router Agent] Task output: https://azure.microsoft.com/en-in/get-started/


 [DEBUG]: == Working Agent: Senior Support Representative
 [INFO]: == Starting Task: XYZ Corporation just reached out with a super important ask:
How to set up up a Windows virtual machine in Azure using the Azure portal?

Anusha from XYZ Corporation is the one that reached out. Use the documentation from the URL provided by the routing_task to find the correct answer to the customer's i

Final Answer: To set up a Windows virtual machine in Azure using the Azure portal, you can follow these steps:

1. Create an Azure account if you don't already have one. You can sign up for an Azure free account which includes 20+ popular services free for 12 months, more than 65+ services free always, and USD 200 credit to use in your first 30 days.

2. Once you have your Azure account, sign in to the Azure portal.

3. In the Azure portal, navigate to the Virtual Machines service.

4. Click on the "Add" button to create a new virtual machine.

5. Choose the Windows Server image that you want to use for your virtual machine.

6. Configure the settings for your virtual machine, such as the size, region, networking, and authentication.

7. Review and confirm your settings, then click on the "Create" button to deploy your Windows virtual machine in Azure.

For more detailed guidance and tutorials on creating virtual machines in Azure, you can visit the Azure documentation at https://azure

Final Answer:

Subject: Step-by-Step Guide on Setting Up a Windows Virtual Machine in Azure Using the Azure Portal

Dear Anusha from XYZ Corporation,

Thank you for reaching out to CrewAITech for assistance on setting up a Windows virtual machine in Azure using the Azure portal. Below is a comprehensive step-by-step guide to help you with the setup process:

1. Log in to the Azure Portal:
   - Open your web browser and navigate to portal.azure.com.
   - Enter your credentials to log in to your Azure account.

2. Create a Virtual Machine:
   - Click on the 'Create a resource' button in the Azure portal.
   - Search for 'Windows Server' in the marketplace and select the desired Windows Server version.
   - Click on 'Create' to begin configuring your virtual machine.

3. Configure Virtual Machine Settings:
   - Enter the required information such as VM name, region, resource group, etc.
   - Choose the appropriate size for your VM based on your requirements.
   - Set up networking configu

- Display the final result as Markdown.

In [ ]:
from IPython.display import Markdown
Markdown(result)

Subject: Step-by-Step Guide on Setting Up a Windows Virtual Machine in Azure Using the Azure Portal

Dear Anusha from XYZ Corporation,

Thank you for reaching out to CrewAITech for assistance on setting up a Windows virtual machine in Azure using the Azure portal. Below is a comprehensive step-by-step guide to help you with the setup process:

1. Log in to the Azure Portal:
   - Open your web browser and navigate to portal.azure.com.
   - Enter your credentials to log in to your Azure account.

2. Create a Virtual Machine:
   - Click on the 'Create a resource' button in the Azure portal.
   - Search for 'Windows Server' in the marketplace and select the desired Windows Server version.
   - Click on 'Create' to begin configuring your virtual machine.

3. Configure Virtual Machine Settings:
   - Enter the required information such as VM name, region, resource group, etc.
   - Choose the appropriate size for your VM based on your requirements.
   - Set up networking configuration, storage, and additional features as needed.

4. Connect to the Virtual Machine:
   - Once the VM is created, click on 'Connect' to download the RDP file.
   - Use the RDP file to connect to your Windows virtual machine.

5. Install Software and Configure Settings:
   - Once connected to the VM, install any necessary software or applications.
   - Configure the settings according to your preferences and requirements.

6. Secure the Virtual Machine:
   - Implement security best practices such as setting up firewalls, enabling encryption, and keeping the system updated.

7. Monitor and Manage the Virtual Machine:
   - Use Azure monitoring tools to keep track of the VM's performance and health.
   - Regularly manage and maintain the virtual machine to ensure optimal functionality.

I hope this guide helps you in setting up your Windows virtual machine in Azure successfully. If you encounter any issues or have further questions, please feel free to reach out to us for assistance.

Best Regards,
[Your Name]
Senior Support Representative
CrewAITech

In [ ]:
inputs2 = {
    "customer": "XYZ Corporation",
    "person": "Anusha",
    "inquiry": "How do I install and configure the AWS CLI?"
}
result2 = crew.kickoff(inputs=inputs2)

 [DEBUG]: == Working Agent: Router Agent
 [INFO]: == Starting Task: Given this user inquiry within single quotes: 'How do I install and configure the AWS CLI?', determine whether it relates to Azure or AWS. Return the best documentation URL: either 'https://azure.microsoft.com/en-in/get-started/' or 'https://docs.aws.amazon.com/'.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 
Based on the user inquiry 'How do I install and configure the AWS CLI?', it is clear that the question is related to AWS rather than Azure. Therefore, the best documentation URL to provide is 'https://docs.aws.amazon.com/'. This URL will direct the user to the official Amazon Web Services documentation on how to install and configure the AWS CLI, providing them with detailed and accurate information to help them with their query.

> Finished chain.
 [DEBUG]: == [Router Agent] Task output: Based on the user inquiry 'How do I install and configure the AWS CLI?', it is clea

Thought: I need to review the response drafted by the Senior Support Representative for XYZ Corporation's inquiry about installing and configuring the AWS CLI.

Action: Ask question to co-worker
Action Input: {"coworker": "Senior Support Representative", "question": "Can you provide details on how the AWS CLI installation and configuration steps were derived? Were they sourced from the official Amazon Web Services documentation?", "context": "Need to verify the accuracy and sources used in the response for Anusha from XYZ Corporation."}

> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 
The installation and configuration steps for the AWS CLI were derived from the official Amazon Web Services documentation. The steps were sourced directly from the AWS Command Line Interface User Guide, which is the official source of information provided by Amazon Web Services for installing and configuring the AWS CLI. This ensures the accuracy and reliability of 

In [ ]:
from IPython.display import Markdown
Markdown(result2)

Hello Anusha,

Thank you for reaching out to us regarding the installation and configuration of AWS CLI. I understand the importance of this task for XYZ Corporation, and I am here to provide you with a comprehensive guide to help you with the process.

To install the AWS CLI, you can follow the steps below:

1. First, you need to ensure that you have Python installed on your system. AWS CLI requires Python 2 version 2.6.5+ or Python 3 version 3.3+. You can check the Python version by running the command `python --version` or `python3 --version` in your terminal.

2. Next, you can install the AWS CLI using pip, the Python package manager. Run the following command in your terminal:
   ```
   pip install awscli
   ```

3. Once the installation is complete, you can verify the installation by running the command `aws --version`. This command should display the AWS CLI version if the installation was successful.

Now, let's move on to the configuration of AWS CLI:

1. To configure the AWS CLI, you can run the following command in your terminal:
   ```
   aws configure
   ```
   This command will prompt you to enter your AWS Access Key ID, AWS Secret Access Key, default region, and default output format. You can obtain these credentials from the AWS Management Console under the IAM (Identity and Access Management) section.

2. After entering the required information, the AWS CLI will create a configuration file at `~/.aws/config` on Unix-based systems or `C:\Users\USERNAME\.aws\config` on Windows systems.

3. You can also customize your AWS CLI configuration by editing the configuration file directly. You can specify profiles, output formats, and other settings in this file.

The installation and configuration steps provided are sourced directly from the official Amazon Web Services documentation, specifically the AWS Command Line Interface User Guide. This ensures the accuracy and reliability of the information. For more detailed information and advanced configuration options, please consult the AWS CLI documentation on the official Amazon Web Services website.

I hope this guide helps you with the installation and configuration of AWS CLI. If you have any further questions or need assistance, please feel free to reach out to me. Thank you for choosing CrewAITech for your support needs.

Best regards,
[Your Name]
Senior Support Representative
CrewAITech

# Conclusion

This agentic RAG project demonstrates how CrewAI agents can collaborate to replicate a structured, multi-step technical support process. By breaking down responsibilities into distinct roles—routing, information retrieval, and quality assurance—the system achieves both accuracy and clarity in responses.

The final output mimics a professionally crafted support email, ensuring that users receive actionable guidance with a human touch. This architecture highlights the potential of agentic systems in building intelligent, context-aware, and role-driven AI support solutions.